The following guide will help us to set all the elements up to have an endpoint capable of doing churn predictions from a model which was trained using BigQuery table.

The table is in the repo <dataset.csv>

requirements: pyarrow

In [11]:
import pandas as pd
from google.cloud import bigquery
from google.cloud import aiplatform

In [8]:
project_id = "vtxdemos"
bq_dataset_name = "demos_us" # dataset demos_us was previously created in the gui
bq_table_name = "churn_bq"
bq_model_name = "churn_bq_model"
table_uri = f"{project_id}.{dataset_name}.{bq_table_name}"
model_uri = f"{project_id}.{dataset_name}.{bq_model_name}"

## BigQuery

In [9]:
bq_client = bigquery.Client(project=project_id)

In [4]:
# Read dataset with pandas

df = pd.read_csv("dataset.csv")

# Load pandas dataframe into bigquery (you have to create a bigquery dataset first)
job = bq_client.load_table_from_dataframe(df, table_uri)
job.result()

/home/jesusarguelles/IdeaProjects/vertex-ai-samples/.venv/lib/python3.13/site-packages/google/cloud/bigquery/_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=vtxdemos, location=US, id=15fae248-9047-491e-9935-b44d2d06d1f8>

In [ ]:
# Create a Regression Model in BigQuery

sql_query = f"""
CREATE OR REPLACE MODEL `{model_uri}`

OPTIONS(
  MODEL_TYPE='LOGISTIC_REG',
  INPUT_LABEL_COLS=['churned'],
  ENABLE_GLOBAL_EXPLAIN=TRUE,
  VERTEX_AI_MODEL_ID="VERTEX_AI_MODEL_ID"
) AS

SELECT
  *
FROM
  `{talbe_uri}`
"""

job = job_client.query(sql_query)
job.result()

## Vertex AI

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="churn_prediction",
)
md=model.deploy(
    deployed_model_display_name="churn_prediction",
    machine_type="n1-standard-4",
    endpoint=endpoint,
    traffic_split={"0": 100}
)

In [ ]:
prediction=md.predict(instances=[
    {
        "user_pseudo_id": "C495AAE4F1CC79CAF4335174F95ABFAC",
        "country": "Mexico",
        "operating_system": "Android",
        "language": "en-ca",
        "cnt_user_engagement": 139,
        "cnt_level_start_quickplay": 33,
        "cnt_level_end_quickplay": 27,
        "cnt_level_complete_quickplay": 11,
        "cnt_level_reset_quickplay": 4,
        "cnt_post_score": 19,
        "cnt_spend_virtual_currency": 3,
        "cnt_ad_reward": 0,
        "cnt_challenge_a_friend": 0,
        "cnt_completed_5_levels": 1,
        "cnt_use_extra_steps": 3,
        "month": 12,
        "julianday": 268,
        "dayofweek": 3
    }
])

In [ ]:
prediction.predictions